In [ ]:
import pandas as pd
import glob

import os
from skimage import io, measure, morphology
import numpy as np

from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm

## Split text by unique word's contours

In [ ]:
img=io.imread('62263_1.jpg')

plt.imshow(img)

In [ ]:
def rgb2smyk(bgr):
    bgrdash = bgr.astype(np.float32)/255.

    # Calculate K as (1 - whatever is biggest out of Rdash, Gdash, Bdash)
    K = 1 - np.max(bgrdash, axis=2)

    # Calculate C
    C = (1-bgrdash[...,2] - K)/(1-K)

    # Calculate M
    M = (1-bgrdash[...,1] - K)/(1-K)

    # Calculate Y
    Y = (1-bgrdash[...,0] - K)/(1-K)

    K*=255
    K=K.astype(np.uint8)

    C*=255
    C=C.astype(np.uint8)

    M*=255
    M=M.astype(np.uint8)

    Y*=255
    Y=Y.astype(np.uint8)

    return C,M,Y,K

In [ ]:
# name='62263_1.jpg'
name='00200_163.jpg'

N_lines=30
height=60

img=io.imread(name)

# for k in range(N_lines):


(C, M, Y, K) = rgb2smyk(img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# thresh=threshold_otsu(Y)
thresh*=0.8
binaryImage=Y>thresh

binaryImage=binaryImage*255
binaryImage=binaryImage.astype(np.uint8)

# binaryImage=morphology.binary_dilation(binaryImage, morphology.disk(1))

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, ksize=(1, 3))

dilated_image = cv2.dilate(binaryImage, kernel)

labeled_image = measure.label(dilated_image)

labels=np.sort(np.unique(labeled_image))
colors=[[np.random.uniform(0,255),np.random.uniform(0,255),np.random.uniform(0,255)] for label in labels]

colored_img=np.zeros((img.shape[0],img.shape[1],3))
for i,line in enumerate(labeled_image):
    for j,label in enumerate(line):
        if label!=0:
            colored_img[i,j]=colors[label]

colored_img=np.array(colored_img,dtype=np.uint8)

fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(20,10))
ax1.imshow(binaryImage, cmap='gray')
ax2.imshow(dilated_image)
ax3.imshow(colored_img)
plt.show()

In [ ]:
# name='62263_1.jpg'
name='00200_163.jpg'
img=io.imread(name)

(C, M, Y, K) = rgb2smyk(img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

thresh*=0.8
binaryImage=Y>thresh

dilated_image=morphology.binary_dilation(binaryImage, morphology.disk(3))

dilated_image=dilated_image*255
dilated_image=dilated_image.astype(np.uint8)


edges = cv2.Canny(dilated_image, 0, 255, L2gradient=False)

# направление обхода контура по часовой стрелке
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
new_contours = []
for cnt in contours:
    new_contours.append(np.array(cnt).reshape((-1, 2)))

labeled_image = measure.label(edges)

labels=np.sort(np.unique(labeled_image))
colors=[[np.random.uniform(0,255),np.random.uniform(0,255),np.random.uniform(0,255)] for label in labels]

colored_img=np.zeros((img.shape[0],img.shape[1],3))
for i,line in enumerate(labeled_image):
    for j,label in enumerate(line):
        if label!=0:
            colored_img[i,j]=colors[label]

colored_img=np.array(colored_img,dtype=np.uint8)

fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(20,10))
ax1.imshow(dilated_image[:1000,:1000], cmap='gray')
ax2.imshow(edges[:1000,:1000], cmap='gray')
ax3.imshow(colored_img[:1000,:1000])
plt.show()

## Slope

In [ ]:
def chaincode(b, unwrap=False):
    '''
    Direction-to-code convention:
    --------------------------
    | deltax | deltay | code |
    |------------------------|    y
    |    0   |   +1   |   2  |    ^     3  2  1
    |    0   |   -1   |   6  |    |      \ | /
    |   -1   |   +1   |   3  |    |   4 -- P -- 0
    |   -1   |   -1   |   5  |    |      / | \ \
    |   +1   |   +1   |   1  |    |     5  6  7
    |   +1   |   -1   |   7  |    |
    |   -1   |    0   |   4  |    +-------------> x
    |   +1   |    0   |   0  |
    --------------------------
    '''
    delta = np.zeros(b.shape, dtype=int)
    delta[:-1, :] = b[1:, :] - b[:-1, :]
    delta[-1, :] = b[0, :] - b[-1, :]

    # check if boundary is 8-connected
    if ((np.abs(delta[:, 0]) > 1) + (np.abs(delta[:, 1]) > 1)).any():
        raise ValueError('Curve is not 8-connected.')

    # check if boundary is close, if so cut last element
    if (np.abs(delta[-1, :]) == 0).all():
        delta = delta[:-1, :]

    if ((np.abs(delta[:, 0]) == 0) * (np.abs(delta[:, 1]) == 0)).any():
        raise ValueError('Curve is degenerate.')

    idx = 3 * delta[:, 0] + delta[:, 1] + 4
    cm = np.array([5, 6, 7, 4, -1, 0, 3, 2, 1])
    cc = cm[idx]

    if unwrap:
        ucc = cc.copy()
        for i in range(1, ucc.shape[0]):
            ucc[i] += 8*np.round((ucc[i-1] - cc[i])/8.)
        return cc, b[0, :], ucc
    else:
        # return cc, b[0, :]
        return cc

In [ ]:
# name='62263_1.jpg'
# name='67205_2.jpg'
# name='00201_73.jpg'
# name='67205_3.jpg'
name='62325_1_12.jpg'
# name='00200_163.jpg'
img=io.imread(name)

(C, M, Y, K) = rgb2smyk(img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

thresh*=0.8
binaryImage=Y>thresh

dilated_image=morphology.binary_dilation(binaryImage, morphology.disk(3))

dilated_image=dilated_image*255
dilated_image=dilated_image.astype(np.uint8)


edges = cv2.Canny(dilated_image, 0, 255, L2gradient=False)

# направление обхода контура по часовой стрелке
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
new_contours = []

for cnt in contours:
    if len(cnt)>5:
        new_contours.append(np.array(cnt).reshape((-1, 2)))

# new_contours=np.array(new_contours)

In [ ]:
# '62325_1_12.jpg'
#  ровный
freeman_chains=[]
for cnt in new_contours:
    freeman_chains.extend(chaincode(cnt))

plt.hist(freeman_chains,bins=8)
plt.show()

In [ ]:
# '00201_73.jpg'
# ровный
freeman_chains=[]
for cnt in new_contours:
    freeman_chains.extend(chaincode(cnt))

plt.hist(freeman_chains,bins=8)
plt.show()

In [ ]:
# '67205_2.jpg'
# наклон вправо
freeman_chains=[]
for cnt in new_contours:
    freeman_chains.extend(chaincode(cnt))

plt.hist(freeman_chains,bins=8)
plt.show()

In [ ]:
# '67205_3.jpg'
# вправо сильно
freeman_chains=[]
for cnt in new_contours:
    freeman_chains.extend(chaincode(cnt))

plt.hist(freeman_chains,bins=8)
plt.show()

## Fractal dimention

In [ ]:
# name='62263_1.jpg'
name='00200_163.jpg'
img=io.imread(name)

(C, M, Y, K) = rgb2smyk(img)

thresh, binaryImage = cv2.threshold(Y, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

thresh*=0.8
binaryImage=Y>thresh

dilated_image=morphology.binary_dilation(binaryImage, morphology.disk(3))

dilated_image=dilated_image*255
dilated_image=dilated_image.astype(np.uint8)


edges = cv2.Canny(dilated_image, 0, 255, L2gradient=False)

# fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(20,10))
# ax1.imshow(dilated_image[:1000,:1000], cmap='gray')
# ax2.imshow(edges[:1000,:1000], cmap='gray')
# # ax3.imshow(colored_img[:1000,:1000])
# plt.show()

In [ ]:
imgs_sums=[]
r=20

name='00200_163.jpg'
img=io.imread(name)

for i in tqdm(range(img.shape[0]//r)):
    for j in range(img.shape[1]//r):
        img_crop=edges[i*r:(i+1)*r,j*r:(j+1)*r]
        img_sum=np.sum(img_crop)
        imgs_sums.append(img_sum)
imgs_sums=np.array(imgs_sums)

fr=np.log(np.where(imgs_sums!=0)[0].shape[0])/np.log(1/r)
print(fr)

5 = -6.5

10 = -4.2

20 = -2.89

